# Basic run of EnergyScope-LCA

In [1]:
import os
import pandas as pd
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing
from utils import get_impact_scores
from new_plots import plot_sankey

In [2]:
res_lcia_max = pd.read_csv('..//02_Model//res_lcia_max.csv')
impact_scores = pd.read_csv('..//03_Results//impact_scores.csv')

In [3]:
# AMPL licence 
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL licence file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [5]:
# Define the solver options
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}

# solver_options = {
#     'solver': 'cplex',
#     'presolve_eps': 5e-9,
#     'presolve_fixeps': 7e-10,
#     'mipgap': 1e-10,
#     'display_eps': 1e-10,
#     'omit_zero_rows': 1,
#     'omit_zero_cols': 1,
#     'show_stats': 1,
#     'solver_msg': 1,
#     'cplex_options': 'integrality=5e-7'
# }

## Total cost optimization

In [4]:
# Initialize the QC model with .mod and .dat files
model = Model(
    mod_files=[
        '..//01_Model//07_Quebec_v1//QC_es_main.mod',
    ],
    dat_files=[
        '..//01_Model//07_Quebec_v1//QC_data.dat',
        '..//01_Model//07_Quebec_v1//QC_mob_techs_dist_B2D.dat',
        '..//01_Model//07_Quebec_v1//QC_techs_B2D.dat',
        '..//01_Model//07_Quebec_v1//QC_mob_params.dat',
        '..//01_Model//07_Quebec_v1//QC_scenarios.dat',
    ],
)

In [6]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [7]:
# Solve the model and get results
results = postprocessing(es.calc())


Presolve eliminates 23937 constraints and 16142 variables.
Adjusted problem:
14121 variables:
	813 binary variables
	13308 linear variables
15870 constraints, all linear; 105218 nonzeros
	6500 equality constraints
	9370 inequality constraints
1 linear objective; 3 nonzeros.

Setting $presolve_fixeps < 4.41e-10 or >= 8.22e-09
could change presolve results.

CPLEX 20.1.0.0: integrality=5e-7
CPLEX solution status 5 with fixed integers:
	optimal with unscaled infeasibilities
CPLEX 20.1.0.0: optimal integer solution; objective 0.6119438245
63 MIP simplex iterations
0 branch-and-bound nodes


### Carbon footprint

In [ ]:
df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
    impact_category=('IMPACT World+ Midpoint 2.0.1_regionalized', 'Midpoint', 'Climate change, short term'),
    df_impact_scores=impact_scores,
    df_results=results,
)

In [ ]:
operation_CF = df_annual_prod['Climate change, short term'].sum()
construction_CF = df_f_mult['Climate change, short term'].sum()
resource_CF = df_annual_res['Climate change, short term'].sum()
Total_CF_from_impact_scores = operation_CF + construction_CF + resource_CF
print(f'Life-cycle carbon footprint per capita: {round(Total_CF_from_impact_scores / 8874683 / 1e3, 3)} t CO2-eq / capita')
print(f'Percentage due to operation: {round(operation_CF / Total_CF_from_impact_scores * 100, 2)} %')
print(f'Percentage due to construction: {round(construction_CF / Total_CF_from_impact_scores * 100, 2)} %')
print(f'Percentage due to resource: {round(resource_CF / Total_CF_from_impact_scores * 100, 2)} %')